## 산탄데르 고객 만족 예측

https://www.kaggle.com/competitions/santander-customer-satisfaction/data

산탄데르 은행이 캐글에 경연을 의뢰한 데이터, 피처 이름은 모두 익명 처리되어 이름만을 가지고 어떤 속성인지 추정할 수 없다. 370개의 피처로 주어진 데이터 세트 기반에서 고객 만족 여부를 예측해야 한다.

TARGET 컬럼: 0 -> 만족 , 1 -> 불만족

- XGBoost, LightGBM 활용 

## 데이터 전처리

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import warnings

In [27]:
warnings.filterwarnings('ignore')
cust_df = pd.read_csv('/Users/pakeunseo/Desktop/2024-2/multicampus/py_ml_env/data/santander-customer-satisfaction/train.csv', encoding= 'latin-1')
print('data shape :', cust_df.shape)
cust_df.head(3)

data shape : (76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


In [28]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [29]:
#불만족 비율 파악 
print(cust_df['TARGET'].value_counts())
unsatisfied_cnt= cust_df[cust_df['TARGET']==1].TARGET.count()
total_cnt = cust_df.TARGET.count()
print('불만족 비율: {0:.2f}'.format(unsatisfied_cnt / total_cnt))

0    73012
1     3008
Name: TARGET, dtype: int64
불만족 비율: 0.04


In [30]:
cust_df.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [31]:
# var3의 값중 -999999값이 116개나 있어 이를 최빈도 값인 2로 변경 
print(cust_df.var3.value_counts()[:10])
cust_df['var3'].replace(-999999,2,inplace= True)

 2         74165
 8           138
-999999      116
 9           110
 3           108
 1           105
 13           98
 7            97
 4            86
 12           85
Name: var3, dtype: int64


In [32]:
#ID 피처 제거 
cust_df.drop('ID', axis=1, inplace= True)

In [45]:
#피처 세트, 레이블 세트 분리
X_features = cust_df.iloc[:,:-1]
y_labels = cust_df.iloc[:,-1]

print('피처 데이터 shape: {0}'.format(X_features.shape))

피처 데이터 shape: (76020, 369)


In [49]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size= 0.2, random_state= 11)

train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트 shape:{0}, 테스트 세트 Shape:{1}'.format(X_train.shape, X_test.shape))

print('학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_cnt)

학습 세트 shape:(60816, 369), 테스트 세트 Shape:(15204, 369)
학습 세트 레이블 값 분포 비율
0    0.960389
1    0.039611
Name: TARGET, dtype: float64

테스트 세트 레이블 값 분포 비율
0    0.960602
1    0.039398
Name: TARGET, dtype: float64


In [51]:
# XGBoost의 early stopping의 검증 데이터 세트로 사용하기 위해 
# X_train, y_train을 다시 학습과 검증 데이터 세트로 분리 
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size= 0.3, random_state= 11)

## XGBoost 모델 학습과 하이퍼 파라미터 튜닝

In [58]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# 모델 생성
xgb_clf = XGBClassifier(n_estimators=500, learning_rate=0.05, random_state=156)

# 모델 학습
xgb_clf.fit(X_tr, y_tr, 
            early_stopping_rounds=100, 
            eval_metric='auc', 
            eval_set=[(X_tr, y_tr), (X_val, y_val)])


# 테스트 데이터에서 1 클래스에 대한 확률 예측
# ROC AUC 점수 계산
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.82085	validation_1-auc:0.79314
[1]	validation_0-auc:0.82587	validation_1-auc:0.79861
[2]	validation_0-auc:0.82692	validation_1-auc:0.79956
[3]	validation_0-auc:0.82928	validation_1-auc:0.80122
[4]	validation_0-auc:0.83038	validation_1-auc:0.80243
[5]	validation_0-auc:0.83314	validation_1-auc:0.80555
[6]	validation_0-auc:0.83541	validation_1-auc:0.80581
[7]	validation_0-auc:0.83546	validation_1-auc:0.80565
[8]	validation_0-auc:0.83692	validation_1-auc:0.80710
[9]	validation_0-auc:0.83744	validation_1-auc:0.80835
[10]	validation_0-auc:0.83867	validation_1-auc:0.80952
[11]	validation_0-auc:0.84010	validation_1-auc:0.81187
[12]	validation_0-auc:0.84067	validation_1-auc:0.81194
[13]	validation_0-auc:0.84183	validation_1-auc:0.81210
[14]	validation_0-auc:0.84196	validation_1-auc:0.81229
[15]	validation_0-auc:0.84231	validation_1-auc:0.81211
[16]	validation_0-auc:0.84298	validation_1-auc:0.81277
[17]	validation_0-auc:0.84364	validation_1-auc:0.81337
[18]	validation_0-au